In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

In [3]:

data = pd.read_csv('/content/Heart_Attack_Data_Set.csv')
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:
# Set random seeds for reproducibility
seed_value = 42
tf.random.set_seed(seed_value)
np.random.seed(seed_value)

In [5]:

# Split the dataset
X = data.drop('target', axis=1)
y = data['target']

In [6]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=2)

In [7]:
# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
#  ANN model with fixed seeds
ANN = keras.Sequential([
    layers.Input(shape=(13,)),
    layers.Dense(64, activation='relu', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=seed_value)),
    layers.Dropout(0.5),  
    layers.Dense(32, activation='relu', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=seed_value)),
    layers.Dropout(0.5),  
    layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=seed_value))
])




In [9]:
# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Compile the model
ANN.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [10]:
# Train the model with early stopping
ANN.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

# Make predictions
y_pred = ANN.predict(X_test)
y_pred = (y_pred > 0.5)  # Convert probabilities to binary values

# Calculate evaluation metrics
accuracy_ANN = accuracy_score(y_test, y_pred)
precision_ANN = precision_score(y_test, y_pred)
recall_ANN = recall_score(y_test, y_pred)
f1_ANN = f1_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)

from sklearn.metrics import roc_curve, auc 
from sklearn.metrics import roc_auc_score


y_pred_prob_ANN = ANN.predict(X_test).ravel()
auc_roc_ANN = roc_auc_score(y_test, y_pred_prob_ANN)
print(f'ANN AUC: {auc_roc_ANN}')


print(f'Accuracy: {accuracy_ANN:.2f}')
print(f'Precision: {precision_ANN:.2f}')
print(f'Recall: {recall_ANN:.2f}')
print(f'F1 Score: {f1_ANN:.2f}')
print('Confusion Matrix:')
print(confusion)

Epoch 1/100
8/8 [==============================] - 2s 41ms/step - loss: 0.7908 - accuracy: 0.4221 - val_loss: 0.7720 - val_accuracy: 0.3929
Epoch 2/100
8/8 [==============================] - 0s 8ms/step - loss: 0.7491 - accuracy: 0.5164 - val_loss: 0.7026 - val_accuracy: 0.5714
Epoch 3/100
8/8 [==============================] - 0s 8ms/step - loss: 0.7079 - accuracy: 0.5697 - val_loss: 0.6513 - val_accuracy: 0.6786
Epoch 4/100
8/8 [==============================] - 0s 10ms/step - loss: 0.6724 - accuracy: 0.5779 - val_loss: 0.6093 - val_accuracy: 0.7500
Epoch 5/100
8/8 [==============================] - 0s 12ms/step - loss: 0.6455 - accuracy: 0.6557 - val_loss: 0.5795 - val_accuracy: 0.7857
Epoch 6/100
8/8 [==============================] - 0s 8ms/step - loss: 0.6023 - accuracy: 0.6475 - val_loss: 0.5559 - val_accuracy: 0.8214
Epoch 7/100
8/8 [==============================] - 0s 7ms/step - loss: 0.5849 - accuracy: 0.6803 - val_loss: 0.5286 - val_accuracy: 0.8571
Epoch 8/100
8/8 [=======